In [1]:
%run "/content/drive/MyDrive/Colab Notebooks/Deep-learning-challenge/AlphabetSoupCharity_initialModel.ipynb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                1100      
                                                                 
 dense_1 (Dense)             (None, 20)                520       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,641
Trainable params: 1,641
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5840 - accuracy: 0.7116
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5579 - accu

In [2]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model2 = tf.keras.models.Sequential()
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['sigmoid','tanh','selu'])
    # Allow kerastuner to decide number of neurons in first layer
    nn_model2.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=90,
        step=5), activation=activation, input_dim=43))
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model2.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=45,
            step=5),
            activation=activation))
        
    nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    # Compile the model
    nn_model2.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model2

In [3]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.4 MB/s eta 0:00:00


In [4]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [5]:
# Run the kerastuner search for best hyperparameters. Takes at least 1.5 hours!
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 02m 24s]
val_accuracy: 0.7269970774650574

Best val_accuracy So Far: 0.7303789854049683
Total elapsed time: 01h 38m 54s


In [6]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'selu', 'first_units': 36, 'num_layers': 6, 'units_0': 26, 'units_1': 36, 'units_2': 41, 'units_3': 26, 'units_4': 1, 'units_5': 26, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0081'}
{'activation': 'selu', 'first_units': 36, 'num_layers': 6, 'units_0': 26, 'units_1': 36, 'units_2': 41, 'units_3': 26, 'units_4': 1, 'units_5': 26, 'tuner/epochs': 17, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 11, 'num_layers': 4, 'units_0': 36, 'units_1': 36, 'units_2': 26, 'units_3': 11, 'units_4': 1, 'units_5': 11, 'tuner/epochs': 50, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [7]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5499 - accuracy: 0.7304 - 1s/epoch - 4ms/step
Loss: 0.5499333143234253, Accuracy: 0.7303789854049683
268/268 - 1s - loss: 0.5544 - accuracy: 0.7301 - 641ms/epoch - 2ms/step
Loss: 0.5543621778488159, Accuracy: 0.7301457524299622
268/268 - 1s - loss: 0.5505 - accuracy: 0.7294 - 676ms/epoch - 3ms/step
Loss: 0.5504990816116333, Accuracy: 0.7294460535049438


In [8]:
# Get the best model hyperparameters
first_hyper = tuner.get_best_hyperparameters(1)[0]
first_hyper.values

{'activation': 'selu',
 'first_units': 36,
 'num_layers': 6,
 'units_0': 26,
 'units_1': 36,
 'units_2': 41,
 'units_3': 26,
 'units_4': 1,
 'units_5': 26,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 17,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0081'}

In [9]:
# Export our model to HDF5 file
tf.keras.models.save_model(nn_model, 'AlphabetSoupCharity_SECONDoptimization_HP.h5')